In [80]:
from functions import *
import json

In [81]:
def layer_alterator(
    vector_mask_path,
    rules_path,
    ucps_folder,
    fractions_folder,
    output_folder,
    layer_name=None
):
    """
    Execute full layer alteration workflow based on vector mask and rule definitions.

    This function:
    - Loads a vector mask (GPKG, GeoJSON, Shapefile)
    - Loads rules from a JSON file
    - Validates and parses the rule configuration
    - Applies masking (C1) or percentage adjustment (C2, C3)
    - Verifies IMD ≥ BSF constraint if relevant
    - Returns a processing summary

    Parameters:
    -----------
    vector_mask_path : str
        Path to the vector file (.gpkg, .geojson, .shp).

    rules_path : str
        Path to a JSON file defining raster layer rules.

    ucp_folder : str
        Folder containing UCP raster files.

    fractions_folder : str
        Folder containing fraction raster files (e.g., "F_AC.tif").

    output_folder : str
        Folder where output rasters will be written.

    layer_name : str, optional
        GPKG layer name if using a .gpkg file.

    Returns:
    --------
    dict
        Summary dictionary containing:
        - rule_id: Applied rule code
        - masking_applied: bool
        - pct_applied: bool
        - imd_bsf_check: str ("passed", "failed", or "skipped")
    """

    summary = {
        "rule_id": None,
        "masking_applied": False,
        "pct_applied": False,
        "imd_bsf_check": "skipped"
    }

    # Step 1: Load vector mask
    gdf_mask = load_vector_mask(vector_mask_path, layer_name)

    # Step 2: Load JSON rules
    with open(rules_path) as f:
        rules = json.load(f)

    # Step 3: Parse and validate rules
    try:
        rule_id, _ = parse_rules_from_mask(gdf_mask, rules)
        summary["rule_id"] = rule_id
    except ValueError as e:
        print("Rule validation failed:", e)
        raise

    # Step 4: Apply rule-based processing
    if rule_id == "C1":
        print("Applying masking...")
        apply_mask_rule_all(gdf_mask, rules, ucps_folder, fractions_folder, output_folder)
        summary["masking_applied"] = True

    elif rule_id in {"C2", "C3"}:
        print("Applying percentage-based adjustments...")
        apply_pct_all(gdf_mask, rules, ucps_folder, fractions_folder, output_folder)
        summary["pct_applied"] = True

        # Step 5: Consistency check (IMD ≥ BSF)
        print("Running IMD ≥ BSF consistency check...")
        try:
            check_imd_bsf_consistency(output_folder)
            summary["imd_bsf_check"] = "passed"
        except Exception as e:
            print("Consistency check failed:", e)
            summary["imd_bsf_check"] = "failed"

    else:
        print("No processing applied (rule:", rule_id, ")")

    return summary

In [82]:
summary = layer_alterator(
    vector_mask_path = "./input_data_for_layer_alterator/vector/vector_pct.geojson",
    rules_path = "./input_data_for_layer_alterator/operation_rules/operation_rules_C3.json",
    ucps_folder = "./input_data_for_layer_alterator/ucps",
    fractions_folder = "./input_data_for_layer_alterator/lc_fractions",
    output_folder = "./output"
)

Rule C3: Layers set to PCT and NONE. Proceeding with PCT logic and treating NONE as 0.
Applying percentage-based adjustments...
F_AC none 0.0 1.0
F_S pct 0.0 1.0
F_M none 0.0 1.0
F_BS pct -30 0.7
F_G pct -30 0.7
F_TV pct 30 1.3
F_W none 0.0 1.0
Pixel sum stats: 0.99999976 1.0000002 1.0
Finished applying percentage changes and polygon-wise normalization to fraction layers.
No explicit NoData value defined in the input raster.
TCH pct 50 1.5
IMD none 0.0 1.0
No explicit NoData value defined in the input raster.
BH none 0.0 1.0
BSF none 0.0 1.0
No explicit NoData value defined in the input raster.
SVF none 0.0 1.0
Running IMD ≥ BSF consistency check...


In [79]:
print(summary)

{'rule_id': 'C3', 'masking_applied': False, 'pct_applied': True, 'imd_bsf_check': 'passed'}


---------------------------------------------------------

In [ ]:
# --- E) Test new IMD and BSF consistency
# check_imd_bsf_consistency(output_folder)

Notes for PCT:

- Think about how to deal with 0 data -> **see "zero_handling" option**
- Test if normalisation [0-1] for each UCP layer is a good proxy to obtain a proportional increase of the UCP that is wanted to be modified by having a final UCP raster still in the range [0-1]? Maybe not, because this changes the whole range of values in the raster -> **Consider only "clip" option for the UCPs**
- Check consistency in IMD and BSF -> **Creates a separate functions "check_imd_bsf_consistency" for checking**
- Think about how to deal with no-data -> **Added additional masking in bot ucp and fractions processing**
- Test if a second normalisation [0-1] per pixel fractions array (other than a normalisation on each single raster) on the modified fractions raster is sufficient to preserve proportion by obeying the rules that sum per pixel = 1 and fractions value >=0. Maybe it's better first to normalise per pixel and then apply the second normalisation to each final raster -> **Applied a single normalization within each pixel in the mask as proportional scaling Fx/sum(all_F)**